# FFI analysis -- for zero points

November 2019

The goal of this notebook is to get zero points for the M67 K2 campaigns, so we can register their relative fluxes.  We will use the Full Frame Images (FFIs).

In [1]:
import numpy as np
import sep
import pandas as pd
import fitsio

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
rcParams['figure.figsize'] = [10., 8.]

In [3]:
from astropy.wcs import WCS
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
Vizier.ROW_LIMIT = 50000

from matplotlib.patches import Ellipse

In [4]:
import sys
import os
sys.path.append(os.path.abspath("../code/"))
import ffi_tools

#### Setup

In [5]:
c5_ffi_fn = '/Volumes/Truro/ffi/ktwo2015127093352-c05_ffi-cal.fits'
c16_ffi_fn = '/Volumes/Truro/ffi/ktwo2017344214411-c16_ffi-cal.fits'
c18_ffi_fn = '/Volumes/Truro/ffi/ktwo2018143080022-c18_ffi-cal.fits'

In [6]:
fn_dict = {5: c5_ffi_fn,
           16: c16_ffi_fn,
           18: c18_ffi_fn}

In [7]:
ffi_ext_dict = {18:13, 16:52, 5:13}

#### Processing

In [8]:
CAMPAIGN = 18

In [9]:
data, hdr = fitsio.read(fn_dict[CAMPAIGN], ext=ffi_ext_dict[CAMPAIGN], header=True)

In [10]:
data = ffi_tools.preprocess_dFFI(data) 
data_sub, bkg = ffi_tools.background_subtract(data, return_bkg=True)
objects = sep.extract(data_sub, 8.0, filter_kernel=None,  err=bkg.globalrms)

In [11]:
angle_degrees = objects['theta'] * 180. / np.pi
twisty = (angle_degrees > -60) & (angle_degrees < 40) & ( objects['a']/objects['b'] > 1.3)
poor_fits = (objects['a'] > 2.0) | (objects['flux'] > 1e6) | (objects['flag'] > 0)

In [12]:
df = pd.DataFrame(objects[~poor_fits & ~twisty])

In [13]:
wcs_hdr = WCS(header=hdr)

In [14]:
xy_in = np.vstack((df.x.values+12, df.y.values+19)).T

In [15]:
ra_decs = wcs_hdr.all_pix2world(xy_in, 0)
df['ra'], df['dec'] = ra_decs[:,0], ra_decs[:,1]

### Cross match with Gaia

It might be helpful to compare the Gaia fluxes as an external calibration source.

In [16]:
c1 = SkyCoord(df['ra'].mean(), df['dec'].mean(), unit=(u.deg, u.deg))

Query Gaia DR2 programmatically.

In [17]:
result = Vizier.query_region(c1, catalog=["I/345/gaia2"], radius=3000.0*u.arcsec)["I/345/gaia2"]

In [18]:
bright_mask = (result['Gmag'] < 20.0).data
result = result[bright_mask]
radecs_gaia = np.vstack([result['RA_ICRS'].data.data, result['DE_ICRS'].data.data]).T

In [19]:
df_gaia = result.to_pandas()

Do the cross matching.

In [20]:
c = SkyCoord(ra=ra_decs[:,0]*u.degree, dec=ra_decs[:,1]*u.degree)
catalog = SkyCoord(ra=radecs_gaia[:, 0]*u.degree, dec=radecs_gaia[:, 1]*u.degree)
idx, d2d, d3d = c.match_to_catalog_sky(catalog)

In [21]:
df['idx'] = idx
df['d2d'] = d2d.to(u.arcsecond).value

In [22]:
df_merge = pd.merge(df, df_gaia, how='left', left_on='idx', right_index=True)

Eliminate bad matches separated by more than 5 arcseconds.

In [23]:
df_merge = df_merge[df_merge.d2d<5]

Save to a csv file.

In [24]:
keep_cols = ['npix', 'xmin', 'xmax', 'ymin', 'ymax', 'x', 'y',
       'x2', 'y2', 'xy', 'a', 'b', 'theta', 'flux', 'peak',
       'xpeak', 'ypeak', 'flag', 'ra', 'dec', 'd2d', 'RA_ICRS',
       'e_RA_ICRS', 'DE_ICRS', 'e_DE_ICRS', 'Source', 'Plx', 'e_Plx', 'pmRA',
       'e_pmRA', 'pmDE', 'e_pmDE', 'Dup', 'FG', 'e_FG', 'Gmag', 'e_Gmag',
       'BP-RP', 'E_BP-RP_']

In [25]:
df_merge[keep_cols].to_csv('../data/K2/FFI_calibration/FFI_C{:02d}_sep_Gaia.csv'.format(CAMPAIGN), index=False)